In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

import sys, os, gc, types
import time
from subprocess import check_output

import sklearn
from sklearn.model_selection import train_test_split

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/data/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_df.csv
key_1.csv
median_49.csv
page_df.csv
page_ohe.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [3]:
train = pd.read_csv(root + 'train_1.csv')
train.fillna(0, inplace = True)
date_df = pd.read_csv(root + 'date_df.csv')
page_df = pd.read_csv(root + 'page_df.csv')
median_49 = pd.read_csv(root + 'median_49.csv')

In [4]:
train_df = train.melt(id_vars=['Page'], var_name='date')

# construct ABT

In [5]:
train_df = train_df.merge(page_df, how='left', on='Page')
train_df = train_df.merge(date_df.drop('date_str', axis = 1), how='left', on='date')

In [6]:
val_days = 62
for i in range(-val_days, 0):
    median_49.iloc[:, i] = median_49.iloc[:, -val_days-1].reshape((len(median_49),1))

/Users/jiayou/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


In [7]:
median_df = median_49.melt(id_vars=['Page'], var_name='date')
median_df.rename(columns={'value':'median'}, inplace = True)
train_df = train_df.merge(median_df, how='left', on=['Page','date'])

In [8]:
train_df['isval'] = (train_df.dayofyear > 366 - val_days) & (train_df.year == 2016)

In [9]:
train_df.head()

,Page,date,value,domain,access,agent,dayofweek,dayofmonth,dayofyear,month,year,isweekday,median,isval
0,2NE1_zh.wikipedia.org_all-access_spider,2015-07-01,18.0,zh.wikipedia.org,all-access,spider,2,1,182,7,2015,True,18.0,False
1,2PM_zh.wikipedia.org_all-access_spider,2015-07-01,11.0,zh.wikipedia.org,all-access,spider,2,1,182,7,2015,True,11.0,False
2,3C_zh.wikipedia.org_all-access_spider,2015-07-01,1.0,zh.wikipedia.org,all-access,spider,2,1,182,7,2015,True,1.0,False
3,4minute_zh.wikipedia.org_all-access_spider,2015-07-01,35.0,zh.wikipedia.org,all-access,spider,2,1,182,7,2015,True,35.0,False
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,2015-07-01,0.0,zh.wikipedia.org,all-access,spider,2,1,182,7,2015,True,0.0,False


In [ ]:
name = 'v12-r0'
down_sample = None
test_size = 0.2
num_searches = 1
boosting_rounds = 10000
stopping_rounds = 10
aug = True